In [1]:
import heapq #El módulo heapq implementa colas de prioridad (heaps)

In [2]:
class Node:
    def __init__(self, position, parent=None, path_cost=0, action=None, used_portals=None):
        self.position = position
        self.parent = parent
        self.path_cost = path_cost
        self.action = action  # Acción que llevó a este nodo
        self.used_portals = set() if used_portals is None else set(used_portals)

    def __lt__(self, other):
        return self.path_cost < other.path_cost

class Problem:
    def __init__(self, maze, start, goals, portales=None):
        self.maze = maze
        self.start = start
        self.goals = goals
        self.portales = portales or {}  # {(x, y): (destino, id_portal)}
        self.actions = {
            (-1, 0): 'Up',
            (1, 0): 'Down',
            (0, -1): 'Left',
            (0, 1): 'Right'
        }

    def is_goal(self, state):
        return state in self.goals

    def valid_position(self, pos):
        rows = len(self.maze)
        cols = len(self.maze[0])
        r, c = pos
        return 0 <= r < rows and 0 <= c < cols and self.maze[r][c] != '#'

    def get_neighbors(self, pos, used_portals):
        neighbors = []
        for move, action in self.actions.items():
            neighbor = (pos[0] + move[0], pos[1] + move[1])
            if self.valid_position(neighbor):
                neighbors.append((neighbor, action, used_portals.copy()))
        # Portales bilaterales, solo si no se ha usado en este camino
        if pos in self.portales:
            destino, id_portal = self.portales[pos]
            if id_portal not in used_portals:
                new_used = used_portals.copy()
                new_used.add(id_portal)
                neighbors.append((destino, f'Portal {id_portal}', new_used))
        # También permitir usar el portal desde el otro extremo (bilateral)
        for entrada, (salida, id_portal) in self.portales.items():
            if pos == salida and id_portal not in used_portals:
                new_used = used_portals.copy()
                new_used.add(id_portal)
                neighbors.append((entrada, f'Portal {id_portal}', new_used))
        return neighbors

In [3]:
def reconstruct_path(node):
    path = []
    actions = []
    while node:
        path.append(node.position)
        actions.append(node.action)
        node = node.parent
    path.reverse()
    actions.reverse()
    return list(zip(path, actions))

In [4]:
def manhattan_distance(pos, goal):
    return abs(pos[0] - goal[0]) + abs(pos[1] - goal[1])

def manhattan_distance_multi(pos, goals):
    return min(abs(pos[0] - g[0]) + abs(pos[1] - g[1]) for g in goals)

def find_all_exits(maze, portales=None):
    # Buscar posiciones de inicio y todas las metas
    start = None
    goals = []
    for i, row in enumerate(maze):
        for j, val in enumerate(row):
            if val == 'S':
                start = (i, j)
            if val == 'E':
                goals.append((i, j))

    if start is None or not goals:
        return []

    problem = Problem(maze, start, goals, portales)

    start_node = Node(start, path_cost=0, action=None, used_portals=set())
    frontier = [(0, start_node)]
    heapq.heapify(frontier)
    reached = {(start, frozenset()): start_node}
    found_paths = {}

    while frontier:
        _, node = heapq.heappop(frontier)
        if problem.is_goal(node.position):
            key = (node.position, frozenset(node.used_portals))
            if key not in found_paths or node.path_cost < found_paths[key][0]:
                found_paths[key] = (node.path_cost, reconstruct_path(node))
            # No return aquí, seguimos buscando otros caminos

        for neighbor_pos, action, new_used_portals in problem.get_neighbors(node.position, node.used_portals):
            new_cost = node.path_cost + 1
            key = (neighbor_pos, frozenset(new_used_portals))
            if key not in reached or new_cost < reached[key].path_cost:
                reached[key] = Node(neighbor_pos, parent=node, path_cost=new_cost, action=action, used_portals=new_used_portals)
                priority = new_cost + manhattan_distance_multi(neighbor_pos, goals)
                heapq.heappush(frontier, (priority, reached[key]))

    # Devolver todos los caminos encontrados a cada meta
    return [(goal, path) for (goal, _), (cost, path) in found_paths.items()]

In [ ]:
maze = [
    ["#", "#", "#", "#", "#", "#", "#", "#", "#", "#"],
    ["#", "S", " ", " ", "#", " ", " ", " ", "#", "#"],
    ["#", " ", "#", " ", "#", " ", "#", " ", " ", "#"],
    ["#", " ", "#", " ", " ", " ", "#", " ", "#", "#"],
    ["#", " ", "#", "#", "#", "@1", "#", " ", " ", "#"],
    ["#", " ", " ", " ", "#", " ", " ", " ", "#", "#"],
    ["#", "#", "#", " ", "#", "#", "#", " ", "#", "#"],
    ["#", "E", "#", " ", " ", " ", "#", " ", "@1", "#"],
    ["#", " ", " ", " ", "#", " ", "#", " ", "E", "#"],
    ["#", "#", "#", "#", "#", "#", "#", "#", "#", "#"]
]

# Definir portales por pares identificados
portales = {}
# Buscar portales y sus pares
portal_entradas = {}
portal_salidas = {}
for i, row in enumerate(maze):
    for j, val in enumerate(row):
        if isinstance(val, str) and val.startswith('@'):
            if val[-1].isdigit():
                id_portal = val[-1]
                if (i, j) not in portal_entradas:
                    portal_entradas[(i, j)] = id_portal
                # Buscar la salida correspondiente
                for x, row2 in enumerate(maze):
                    for y, val2 in enumerate(row2):
                        if (x, y) != (i, j) and val2 == val:
                            portal_salidas[id_portal] = (x, y)
# Crear diccionario de portales: entrada -> (salida, id)
for (i, j), id_portal in portal_entradas.items():
    if id_portal in portal_salidas:
        portales[(i, j)] = (portal_salidas[id_portal], id_portal)

# --- Uso e impresión ---
# Filtrar solo el camino más corto por cada meta
paths = find_all_exits(maze, portales)
best_paths = {}
for goal, path in paths:
    if goal not in best_paths or len(path) < len(best_paths[goal]):
        best_paths[goal] = path
if best_paths:
    for goal, path in best_paths.items():
        print(f"Meta: {goal}")
        print(f"Longitud del camino: {len(path)-1} pasos")
        print("Camino:")
        for step in path:
            print(step)
        print("-"*30)
else:
    print("No se encontró salida")

Meta: (7, 1)
Longitud del camino: 12 pasos
Camino:
((1, 1), None)
((2, 1), 'Down')
((3, 1), 'Down')
((4, 1), 'Down')
((5, 1), 'Down')
((5, 2), 'Right')
((5, 3), 'Right')
((6, 3), 'Down')
((7, 3), 'Down')
((8, 3), 'Down')
((8, 2), 'Left')
((8, 1), 'Left')
((7, 1), 'Up')
------------------------------
Meta: (8, 8)
Longitud del camino: 9 pasos
Camino:
((1, 1), None)
((1, 2), 'Right')
((1, 3), 'Right')
((2, 3), 'Down')
((3, 3), 'Down')
((3, 4), 'Right')
((3, 5), 'Right')
((4, 5), 'Down')
((7, 8), 'Portal 1')
((8, 8), 'Down')
------------------------------
